In [10]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 1024
n_epochs = 200

In [3]:
df = pd.read_csv('options-df-sigma.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['date', 'exdate', 'impl_volatility'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [4]:
call_df.head()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price,sigma_20
0,600.0,28.000,29.000,4,8555,47,5.17,624.22,0.007761
10,475.0,152.875,153.875,0,2700,145,5.12,624.22,0.007761
13,600.0,52.375,53.375,0,2155,327,5.05,624.22,0.007761
17,610.0,20.000,20.750,923,8860,47,5.17,624.22,0.007761
18,675.0,34.000,35.000,0,350,691,5.10,624.22,0.007761


In [5]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        (call_df.best_bid + call_df.best_offer) / 2,
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                    (put_df.best_bid + put_df.best_offer) / 2,
                                                                    test_size=0.01, random_state=42)

In [8]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(LeakyReLU())

model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer=Adam(lr=1e-5))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 400)               3200      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 400)               160400    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 400)               160400    
__________

In [11]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)



Train on 5001241 samples, validate on 50518 samples


Epoch 1/200
5001241/5001241 [==============================] - 47s 9us/step - loss: 5036.4262 - val_loss: 188.0223
Epoch 2/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 176.6153 - val_loss: 142.4848
Epoch 3/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 141.0185 - val_loss: 187.0093
Epoch 4/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 123.1690 - val_loss: 102.7135
Epoch 5/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 112.7073 - val_loss: 166.1797
Epoch 6/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 107.7671 - val_loss: 107.4747
Epoch 7/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 104.1708 - val_loss: 116.1972
Epoch 8/200
5001241/5001241 [==============================] - 34s 7us/step - loss: 102.3607 - val_loss: 88.9234
Epoch 9/200
5001241/5001241 [===

5001241/5001241 [==============================] - 33s 7us/step - loss: 62.6218 - val_loss: 68.7422
Epoch 66/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 62.1148 - val_loss: 75.4922
Epoch 67/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 61.7874 - val_loss: 113.9917
Epoch 68/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 62.1586 - val_loss: 74.0346
Epoch 69/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 61.2773 - val_loss: 59.3865
Epoch 70/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 61.4629 - val_loss: 57.4262
Epoch 71/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 60.5791 - val_loss: 58.6497
Epoch 72/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 60.7140 - val_loss: 64.9375
Epoch 73/200
5001241/5001241 [==============================] - 33s 7us/step - loss: 60.4514 - val_loss: 73.

KeyboardInterrupt: 

In [12]:
model.save('mlp1-100.h5')

In [13]:
call_y_pred = model.predict(call_X_test)

In [22]:
'test set mse', np.mean(np.square(call_y_test - np.reshape(call_y_pred, call_y_pred.shape[0])))

('test set mse', 60.14225438740651)

# Now beyond 100 epochs using 1e-6 lr

In [23]:
model.compile(loss='mse', optimizer=Adam(lr=1e-6))

In [24]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=20, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.6987 - val_loss: 52.2438
Epoch 2/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.6462 - val_loss: 52.2117
Epoch 3/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.6000 - val_loss: 52.1503
Epoch 4/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.5716 - val_loss: 52.5116
Epoch 5/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.5317 - val_loss: 52.7806
Epoch 6/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.5112 - val_loss: 52.1189
Epoch 7/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.5029 - val_loss: 52.6333
Epoch 8/20
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.4770 - val_loss: 52.2363
Epoch 9/20
5001241/5001241 [==============================] 

KeyboardInterrupt: 

In [25]:
model.save('mlp1-110.h5')

In [26]:
call_y_pred2 = model.predict(call_X_test)
'test set mse', np.mean(np.square(call_y_test - np.reshape(call_y_pred2, call_y_pred2.shape[0])))

('test set mse', 53.73220293447192)

# Now 1e-7 lr

In [27]:
model.compile(loss='mse', optimizer=Adam(lr=1e-7))

In [28]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/10
5001241/5001241 [==============================] - 35s 7us/step - loss: 53.1470 - val_loss: 51.8260
Epoch 2/10
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.1357 - val_loss: 51.8296
Epoch 3/10
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.1348 - val_loss: 51.8111
Epoch 4/10
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.1323 - val_loss: 51.8094
Epoch 5/10
5001241/5001241 [==============================] - 34s 7us/step - loss: 53.1289 - val_loss: 51.8534
Epoch 6/10
 152576/5001241 [..............................] - ETA: 38s - loss: 52.8562

KeyboardInterrupt: 

In [29]:
model.save('mlp1-115.h5')
call_y_pred3 = model.predict(call_X_test)
'test set mse', np.mean(np.square(call_y_test - np.reshape(call_y_pred3, call_y_pred3.shape[0])))

('test set mse', 53.5843140261273)

# Now 1e-8 lr

In [30]:
model.compile(loss='mse', optimizer=Adam(lr=1e-8))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/5
5001241/5001241 [==============================] - 37s 7us/step - loss: 53.0968 - val_loss: 51.7894
Epoch 2/5
5001241/5001241 [==============================] - 36s 7us/step - loss: 53.0941 - val_loss: 51.7931
Epoch 3/5
5001241/5001241 [==============================] - 36s 7us/step - loss: 53.0935 - val_loss: 51.7934
Epoch 4/5
5001241/5001241 [==============================] - 37s 7us/step - loss: 53.0932 - val_loss: 51.7975
Epoch 5/5
5001241/5001241 [==============================] - 37s 7us/step - loss: 53.0932 - val_loss: 51.7929


In [31]:
model.save('mlp1-120.h5')
call_y_pred4 = model.predict(call_X_test)
'test set mse', np.mean(np.square(call_y_test - np.reshape(call_y_pred4, call_y_pred4.shape[0])))

('test set mse', 53.54766449036366)